In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
}

In [29]:
url = "https://webmail.iitb.ac.in/?_task=mail&_caps=pdf%3D1%2Cflash%3D0%2Ctiff%3D0%2Cwebp%3D1%2Cpgpmime%3D0&_uid=647&_mbox=INBOX&_action=show"

In [4]:
url = "https://webmail.iitb.ac.in/?_task=mail&_mbox=INBOX"

In [5]:
response = requests.get(url, headers = header)
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content,'html.parser')

In [7]:
soup.h1

In [8]:
response.url

'https://webmail.iitb.ac.in/?_task=mail&_mbox=INBOX'

In [9]:
names = soup.find('div', style="font-family: Verdana,Geneva,sans-serif; font-size: 13.333333px")
print(names)

None


In [10]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <link href="/assets/styles.css" rel="stylesheet" type="text/css"/>
  <title>
   WM3 IITB
  </title>
  <!-- USAGE NOTICE -->
  <!-- You need to set the redirection URL *TWICE* in the body: once for mobile and once for desktop -->
  <!-- Customize colors below -->
  <meta content="#004D40" name="theme-color"/>
  <style>
   :root {
			--theme: #004D40;
			--link-color: white;
			--scrollbar: yellow;
			--border-color: lightgreen;
			--foreground: white;
		}
		.error {  width: 100%; text-align: center; margin: 5px 0; color: red; }
  </style>
 </head>
 <body>
  <div class="bg-image">
  </div>
  <div class="main" id="main" style="opacity: 0">
   <div class="right">
    <div class="logindiv desktop">
     <div class="allcaps sign-in-title">
      Sign in to continue
     </div>
     <!-- <a class="lo

In [32]:
import imaplib
import ssl
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.message import MIMEMessage
from email.parser import BytesParser

# IMAP server credentials
IMAP_SERVER = 'imap.iitb.ac.in'
IMAP_USER = '24m0741'
IMAP_PASSWORD = 'f5c1577b05c2e063c6aea6aecf2de509'
IMAP_PORT = 993

# SMTP server credentials (for sending emails)
SMTP_SERVER = 'smtp-auth.iitb.ac.in'
SMTP_PORT = 587
SMTP_USER = '24m0741'
SMTP_PASSWORD = 'f5c1577b05c2e063c6aea6aecf2de509'

# Target email address to forward to
FORWARD_TO = 'bhagawanlalith@gmail.com'
SENDER_EMAIL = 'gsecha@iitb.ac.in'

# Create an SSL context for IMAP over SSL/TLS (force TLSv1.2 or higher)
imap_ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
imap_ssl_context.options |= ssl.OP_NO_SSLv2 | ssl.OP_NO_SSLv3  # Disable SSLv2/v3 for security
imap_ssl_context.set_ciphers('AES256-GCM-SHA384')  # Example of a strong cipher suite

# Create an SSL context for SMTP over STARTTLS (force TLSv1.2 or higher)
smtp_ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
smtp_ssl_context.options |= ssl.OP_NO_SSLv2 | ssl.OP_NO_SSLv3  # Disable SSLv2/v3 for security
smtp_ssl_context.set_ciphers('ECDHE-RSA-AES128-GCM-SHA256')  # Example of a strong cipher suite


def forward_emails():
    # Connect to the IMAP server using SSL/TLS
    mail = imaplib.IMAP4_SSL(IMAP_SERVER, port=IMAP_PORT, ssl_context=imap_ssl_context)
    mail.login(IMAP_USER, IMAP_PASSWORD)

    mail.select("INBOX")

    # Search for emails from a specific sender
    status, messages = mail.search(None, f'FROM "{SENDER_EMAIL}"')
    
    if status != "OK":
        print("No emails found from the specified sender!")
        
    else:
        print("Emails found:")
        email_ids = messages[0].split()  # Split the byte string into a list of email IDs
        for email_id in email_ids:
            print(f"Email ID: {email_id.decode()}")  # Print each email ID
        
    email_ids = messages[0].split()

    for email_id in email_ids:
        # Fetch the email by ID
        status, data = mail.fetch(email_id, "(RFC822)")

        if status != "OK":
            print(f"Failed to fetch email {email_id}")
            continue

        # Parse the email content
        msg = BytesParser().parsebytes(data[0][1])
        
        # Forward the email
        forward_msg = MIMEMultipart()
        forward_msg['From'] = IMAP_USER
        forward_msg['To'] = FORWARD_TO
        forward_msg['Subject'] = "Fwd: " + msg['Subject']
        
        # Create a MIME message and attach the original email
        forward_msg.attach(MIMEMessage(msg))
        print("Email details:")
        print("From:", forward_msg['From'])
        print("To:", forward_msg['To'])
        print("Subject:", forward_msg['Subject'])
        print("Body:", msg.as_string())

        # Send the email using SMTP with STARTTLS
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls(context=smtp_ssl_context)  # Upgrade the connection to secure using STARTTLS
            smtp.login(SMTP_USER, SMTP_PASSWORD)
            response = smtp.sendmail(IMAP_USER, FORWARD_TO, forward_msg.as_string())
            print(response)
            print(f"Email {email_id} forwarded to {FORWARD_TO}")

    mail.logout()

if __name__ == "__main__":
    print("In main")
    forward_emails()

/tmp/ipykernel_11846/2094970498.py:26: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  imap_ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
/tmp/ipykernel_11846/2094970498.py:31: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  smtp_ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)


In main
Emails found:
Email ID: 22
Email ID: 121
Email ID: 188
Email ID: 275
Email ID: 279
Email ID: 308
Email ID: 432
Email ID: 539
Email ID: 623
Email ID: 648
Email ID: 692
Email ID: 816
Email ID: 817
Email ID: 828
Email ID: 829
Email ID: 864
Email ID: 870
Email ID: 888
Email ID: 896
Email ID: 898
Email ID: 903
Email ID: 935
Email ID: 942
Email ID: 949
Email ID: 987
Email ID: 995
Email ID: 998
Email details:
From: 24m0741
To: bhagawanlalith@gmail.com
Subject: Fwd: [Student-notices] Fwd: Join Abhyuday's Green Earth Campaign on the
 World Environment Day!
Body: Return-Path: <student-notices-bounces@iitb.ac.in>
Delivered-To: 24m0741@iitb.ac.in
Received: from rp17.iitb.ac.in
	by RP17.iitb.ac.in with LMTP
	id cHYsOP4ao2YU2goAXyGcFQ
	(envelope-from <student-notices-bounces@iitb.ac.in>)
	for <24m0741@iitb.ac.in>; Fri, 26 Jul 2024 09:11:50 +0530
Received: from lists.iitb.ac.in (lists.iitb.ac.in [10.200.5.100])
	by rp17.iitb.ac.in (Postfix) with QMQP id E50281800C51
	for <24m0741@iitb.ac.in>;

In [13]:
!python --version

/bin/bash: line 1: python: command not found
